In [1]:
import pandas as pd
import numpy as np
import csv
from enum import Enum

列挙型の定義

In [2]:
class NPerson(Enum):
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4

    @property
    def index(self):
        return self.value - 1

In [3]:
class Region(Enum):
    REGION1 = 1
    REGION2 = 2
    REGION3 = 3
    REGION4 = 4
    REGION5 = 5
    REGION6 = 6
    REGION7 = 7
    REGION8 = 8

    @property
    def index(self) -> int:
        return self.value - 1

給湯需要・入浴回数の読み込み

In [4]:
d_demand_excel = pd.read_excel('dhw_demand.xlsx', sheet_name=None, index_col=0)

def get_schedule(p: NPerson, key_name: str):
    sheet = d_demand_excel[str(p.value)]
    return {
        'HIL': np.array(sheet[key_name + '_HIL']),
        'HIS': np.array(sheet[key_name + '_HIS']),
        'WL': np.array(sheet[key_name + '_WL']),
        'WM': np.array(sheet[key_name + '_WM']),
        'WS': np.array(sheet[key_name + '_WS']),
        'HO': np.array(sheet[key_name + '_HO'])
    }

年間スケジュールの読み込み

In [5]:
schedule_d = None
with open('schedule.csv', encoding='utf8') as f:
    reader = csv.reader(f, delimiter='\t')
    rows = [row for row in reader]
    schedule_d = [
        {
            '休日在宅（大）': 'HIL',
            '休日在宅（小）': 'HIS',
            '平日（大）': 'WL',
            '平日（中）': 'WM',
            '平日（小）': 'WS',
            '休日外出': 'HO'
        }[row[3]] for row in rows[1:366]
    ]
schedule_d = np.array(schedule_d)
print(schedule_d)

['HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'HIS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'HO' 'HO' 'HIS' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS' 'HO' 'WM' 'WS' 'WL' 'WS' 'WM' 'HIS'
 'HIL' 'WM' 'WS' 'WM' 'WM' 'WM' 'HIS

給湯需要・入浴回数の年間展開

In [6]:
def get_x_d_t(p: NPerson, key_name: str):
    x_dict = get_schedule(p=p, key_name=key_name)
    x_d_t = np.zeros(shape=[365, 24], dtype=float)
    x_d_t[schedule_d=='HIL'] = x_dict['HIL']
    x_d_t[schedule_d=='HIS'] = x_dict['HIS']
    x_d_t[schedule_d=='WL'] = x_dict['WL']
    x_d_t[schedule_d=='WM'] = x_dict['WM']
    x_d_t[schedule_d=='WS'] = x_dict['WS']
    x_d_t[schedule_d=='HO'] = x_dict['HO']
    return x_d_t

# 台所需要
def get_w_k_d_t(p: NPerson):
    return get_x_d_t(p=p, key_name='w_k')

# シャワー需要
def get_w_s_d_t(p: NPerson):
    return get_x_d_t(p=p, key_name='w_s')

# 洗面需要
def get_w_w_d_t(p: NPerson):
    return get_x_d_t(p=p, key_name='w_w')

# 湯張り需要
def get_w_b_d_t(p: NPerson):
    return get_x_d_t(p=p, key_name='w_b')

# 入浴回数
def get_n_b_d_t(p: NPerson):
    return get_x_d_t(p=p, key_name='n_b')


外気温度の読み込み

In [7]:
def get_theta_ex_d_t(region: Region):
    df_outdoor = pd.read_csv('outdoor_temp.csv', names=('1','2','3','4','5','6','7','8'))
    theta_ex_r_d_t = (np.array(df_outdoor).T).reshape([8, 365, 24])
    return theta_ex_r_d_t[region.index]

夜間平均外気温度

In [8]:
def get_theta_ex_nave_d(region: Region) -> np.ndarray:
    theta_ex_d_t = get_theta_ex_d_t(region=region)
    return np.average(np.roll(theta_ex_d_t, 1)[:, 0:8], axis=1)

日平均外気温度

In [9]:
def get_theta_ex_ave_d(region: Region) -> np.ndarray:
    theta_ex_d_t = get_theta_ex_d_t(region=region)
    return np.average(theta_ex_d_t, axis=1)

日付dを基準とした期間平均外気温度

In [10]:
def get_theta_ex_prd_ave_d(region: Region) -> np.ndarray:
    theta_ex_ave_d = get_theta_ex_ave_d(region=region)
    return np.convolve(np.concatenate([theta_ex_ave_d[355:], theta_ex_ave_d[:364]]), np.ones(10)/10, mode='valid')

日平均給水温度

In [11]:
def get_theta_wtr_d(region: Region) -> np.ndarray:
    theta_ex_prd_ave_d = get_theta_ex_prd_ave_d(region=region)
    a_wtr = [0.6639, 0.6639, 0.6054, 0.6054, 0.8660, 0.8516, 0.9223, 0.6921][region.index]
    b_wtr = [3.466, 3.466, 4.515, 4.515, 1.665, 2.473, 2.097, 7.167][region.index]
    return np.maximum(a_wtr * theta_ex_prd_ave_d + b_wtr, 0.5)

浴槽沸かし直しによる給湯熱負荷

In [12]:
def get_l_ba_d_t(region: Region, p: NPerson) -> np.ndarray:
    theta_ex_ave_d = get_theta_ex_ave_d(region=region)

    def get_l_ba(theta_ex_ave, p):
        if theta_ex_ave < 7.0:
            a_ba = [0.0, -0.06, -0.10, -0.12][p.index]
            b_ba = [0.0, 3.02, 4.91, 6.00][p.index]
        elif theta_ex_ave < 16.0:
            a_ba = [0.0, -0.06, -0.10, -0.13][p.index]
            b_ba = [0.0, 3.04, 4.93, 6.04][p.index]
        elif theta_ex_ave < 25.0:
            a_ba = [0.0, -0.07, -0.11, -0.14][p.index]
            b_ba = [0.0, 3.13, 5.07, 6.21][p.index]
        else:
            a_ba = [0.0, -0.06, -0.10, -0.12][p.index]
            b_ba = [0.0, 2.92, 4.77, 5.81][p.index]
        return a_ba * theta_ex_ave + b_ba

    n_b_d_t = get_n_b_d_t(p=p)
    n_b_d = n_b_d_t.sum(axis=1)
    n_b_d_dsh = n_b_d
    # ゼロ割を回避するために分母がゼロの場合は9999に置換する
    n_b_d_dsh[n_b_d == 0.0] = 9999.0

    return np.vectorize(get_l_ba)(theta_ex_ave=theta_ex_ave_d, p=p)[:, np.newaxis] * n_b_d_t / n_b_d_dsh[:, np.newaxis]


浴槽自動湯はり時における基準給湯量

In [13]:
def get_w_b2_d_t(p: NPerson):
    return get_w_b_d_t(p=p)

節湯補正給湯量  
今回の基準では節湯措置を見込まないので、削減量はゼロで計算する。  
浴槽水栓湯張りは使わないので定義しない。

In [14]:
def get_w_dsh_k_d_t(p: NPerson):
    return get_w_k_d_t(p=p)

def get_w_dsh_s_d_t(p: NPerson):
    return get_w_s_d_t(p=p)

def get_w_dsh_w_d_t(p: NPerson):
    return get_w_w_d_t(p=p)

def get_w_dsh_b2_d_t(p: NPerson):
    return get_w_b2_d_t(p=p)

基準給湯温度

In [15]:
def get_theta_sw_k():
    return 40.0

def get_theta_sw_s():
    return 40.0

def get_theta_sw_w():
    return 40.0

def get_theta_sw_b2():
    return 40.0


節湯補正給湯熱負荷

In [16]:
def get_l_dsh_k_d_t(region: Region, p: NPerson):
    return get_w_dsh_k_d_t(p=p) * (get_theta_sw_k() - get_theta_wtr_d(region=region))[:, np.newaxis] * 4.186 * 0.001

def get_l_dsh_s_d_t(region: Region, p: NPerson):
    return get_w_dsh_s_d_t(p=p) * (get_theta_sw_s() - get_theta_wtr_d(region=region))[:, np.newaxis] * 4.186 * 0.001

def get_l_dsh_w_d_t(region: Region, p: NPerson):
    return get_w_dsh_w_d_t(p=p) * (get_theta_sw_w() - get_theta_wtr_d(region=region))[:, np.newaxis] * 4.186 * 0.001

def get_l_dsh_b1_d_t():
    return np.zeros(shape=(365, 24), dtype=float)

def get_l_dsh_b2_d_t(region: Region, p: NPerson):
    return get_w_dsh_b2_d_t(p=p) * (get_theta_sw_b2() - get_theta_wtr_d(region=region))[:, np.newaxis] * 4.186 * 0.001

def get_l_dsh_ba1_d_t():
    return np.zeros(shape=(365, 24), dtype=float)

def get_l_dsh_ba2_d_t(region: Region, p: NPerson):
    return get_l_ba_d_t(region=region, p=p) * 1.25

太陽熱補正熱負荷  
太陽熱を今回は見込まないため、節湯補正給湯熱負荷に等しいとする。  
記号はダッシュがつく数が異なるが、読み替えることで対応する。

ガス給湯機・石油給湯機共通

In [17]:
def get_l_dsh_k_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_k_d_t(region=region, p=p), axis=1)

def get_l_dsh_s_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_s_d_t(region=region, p=p), axis=1)

def get_l_dsh_w_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_w_d_t(region=region, p=p), axis=1)

def get_l_dsh_b1_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_b1_d_t(region=region, p=p), axis=1)

def get_l_dsh_b2_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_b2_d_t(region=region, p=p), axis=1)

def get_l_dsh_ba1_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_ba1_d_t(region=region, p=p), axis=1)

def get_l_dsh_ba2_d(region: Region, p: NPerson) -> np.ndarray:
    return np.sum(get_l_dsh_ba2_d_t(region=region, p=p), axis=1)

In [18]:
def get_e_gk_hs_d_t(region: Region, p: NPerson, a_std_u, b_std_u, c_std_u, f_hs):

    # ['k', 's', 'w', 'b1', 'b2', 'ba1', 'ba2']
    a_u = [a_std * f_hs for a_std in a_std_u] 
    b_u = [b_std * f_hs for b_std in b_std_u] 
    c_u = [c_std * f_hs for c_std in c_std_u]

    e_k_d = a_u[0] * get_theta_ex_ave_d(region=region) + b_u[0] * (get_l_dsh_k_d(region=region, p=p) + get_l_dsh_w_d(region=region, p=p)) + c_u[0]
    e_s_d = a_u[1] * get_theta_ex_ave_d(region=region) + b_u[1] * get_l_dsh_s_d(region=region, p=p) + c_u[1]
    e_w_d = a_u[2] * get_theta_ex_ave_d(region=region) + b_u[2] * (get_l_dsh_k_d(region=region, p=p) + get_l_dsh_w_d(region=region, p=p)) + c_u[2]
    e_b2_d = a_u[4] * get_theta_ex_ave_d(region=region) + b_u[4] * get_l_dsh_b2_d(region=region, p=p) + c_u[4]
    e_ba2_d = a_u[6] * get_theta_ex_ave_d(region=region) + b_u[6] * get_l_dsh_ba2_d(region=region, p=p) + c_u[6]

    e_g_hs_d_t \
        = get_l_dsh_k_d_t(region=region, p=p) / e_k_d[:, np.newaxis] \
        + get_l_dsh_s_d_t(region=region, p=p) / e_s_d[:, np.newaxis] \
        + get_l_dsh_w_d_t(region=region, p=p) / e_w_d[:, np.newaxis] \
        + get_l_dsh_b2_d_t(region=region, p=p) / e_b2_d[:, np.newaxis] \
        + get_l_dsh_ba2_d_t(region=region, p=p) / e_ba2_d[:, np.newaxis]
    
    return e_g_hs_d_t


In [19]:
def get_e_e_hs_d_t(region: Region, p: NPerson, eq2, eq4):

    l_dsh_ba2_d_t = get_l_dsh_ba2_d_t(region=region, p=p)
    l_dsh_ba2_d = get_l_dsh_ba2_d(region=region, p=p)
    l_dsh_ba2_d_dsh = l_dsh_ba2_d
    l_dsh_ba2_d_dsh[l_dsh_ba2_d==0.0] = 9999.0
    e_e_hs_aux3_d_t = ((eq4[0] * l_dsh_ba2_d + eq4[1]) * 1000 / 3600 / l_dsh_ba2_d)[:, np.newaxis] * l_dsh_ba2_d_t
    e_e_hs_aux3_d_t[l_dsh_ba2_d==0.0, :] = 0.0

    w_dsh_b2_d_t = get_w_dsh_b2_d_t(p=p)
    w_dsh_b2_d = np.sum(w_dsh_b2_d_t, axis=1)
    w_dsh_b2_d[w_dsh_b2_d==0.0] = 9999.0
    e_e_hs_aux2_d_t = 0.07 * 1000 / 3600 * w_dsh_b2_d_t / w_dsh_b2_d[:, np.newaxis]

    e_e_hs_aux1_d_t = (
        ((eq2[0] * get_theta_ex_ave_d(region=region) + eq2[1]) / 24)[:, np.newaxis]
        + eq2[2] * (get_w_dsh_k_d_t(p=p) + get_w_dsh_s_d_t(p=p) + get_w_dsh_w_d_t(p=p))
    ) * 1000 / 3600

    return e_e_hs_aux1_d_t + e_e_hs_aux2_d_t + e_e_hs_aux3_d_t

In [20]:
def get_e_g_hs_d_t(region: Region, p: NPerson):
    
    e_rtd = 0.782

    e_gk_hs_d_t = get_e_gk_hs_d_t(
        region=region,
        p=p,
        a_std_u=[0.0019, 0.0006, 0.0019, 0.0000, 0.0000, 0.0000, 0.0033],
        b_std_u=[0.0013, 0.0005, 0.0013, 0.0002, -0.0005, 0.0002, 0.0194],
        c_std_u=[0.6533, 0.7414, 0.6533, 0.7839, 0.7828, 0.7839, 0.5776],
        f_hs=(0.8754 * e_rtd + 0.06) / 0.745
    )

    e_e_hs_d_t = get_e_e_hs_d_t(
        region=region,
        p=p,
        eq2=(-0.00172, 0.2822, 0.000393),
        eq4=(0.01723, 0.06099)
    )

    return e_e_hs_d_t * 9.76 + e_gk_hs_d_t


In [21]:
def get_e_k_hs_d_t(region: Region, p: NPerson):
    
    e_rtd = 0.813

    e_gk_hs_d_t = get_e_gk_hs_d_t(
        region=region,
        p=p,
        a_std_u=[0.0005, 0.0024, 0.0005,  0.0000,  0.0000,  0.0000, 0.0062],
        b_std_u=[0.0028, 0.0021, 0.0028, -0.0027, -0.0024, -0.0027, 0.0462],
        c_std_u=[0.6818, 0.7560, 0.6818,  0.9026,  0.8885,  0.9026, 0.4001],
        f_hs=(0.8669 * e_rtd + 0.091) / 0.796
    )

    e_e_hs_d_t = get_e_e_hs_d_t(
        region=region,
        p=p,
        eq2=(-0.00235, 0.3388, 0.000780),
        eq4=(0.02102, 0.12852)
    )

    return e_e_hs_d_t * 9.76 + e_gk_hs_d_t

基準の結果との比較確認

1地域

In [22]:
get_e_k_hs_d_t(region=Region.REGION1, p=NPerson.ONE).sum(), 234.4362 * 30 + 4913.4197

(11949.156874373364, 11946.505700000002)

In [23]:
get_e_k_hs_d_t(region=Region.REGION1, p=NPerson.TWO).sum(), 307.3813 * 60 + 536.7172

(18979.59355465561, 18979.5952)

In [24]:
get_e_k_hs_d_t(region=Region.REGION1, p=NPerson.THREE).sum(), 109.6993 * 90 + 18328.0935

(28201.031730029063, 28201.0305)

In [25]:
get_e_k_hs_d_t(region=Region.REGION1, p=NPerson.FOUR).sum(), 31492.0111

(31492.01112773883, 31492.0111)

2地域

In [26]:
get_e_k_hs_d_t(region=Region.REGION2, p=NPerson.ONE).sum(), 228.3869 * 30 + 4844.5224

(11702.133395666227, 11696.1294)

In [27]:
get_e_k_hs_d_t(region=Region.REGION2, p=NPerson.TWO).sum(), 300.6392 * 60 + 509.3896

(18547.73928113309, 18547.7416)

In [28]:
get_e_k_hs_d_t(region=Region.REGION2, p=NPerson.THREE).sum(), 107.3577 * 90 + 17904.7241

(27566.914123099807, 27566.9171)

In [29]:
get_e_k_hs_d_t(region=Region.REGION2, p=NPerson.FOUR).sum(), 30787.6441

(30787.64413860073, 30787.6441)

3地域

In [30]:
get_e_k_hs_d_t(region=Region.REGION3, p=NPerson.ONE).sum(), 212.0339 * 30 + 4531.3503

(10903.029064166461, 10892.3673)

In [31]:
get_e_k_hs_d_t(region=Region.REGION3, p=NPerson.TWO).sum(), 280.6827 * 60 + 412.4201

(17253.382671995725, 17253.3821)

In [32]:
get_e_k_hs_d_t(region=Region.REGION3, p=NPerson.THREE).sum(), 100.5938 * 90 + 16620.4179

(25673.8639713445, 25673.859900000003)

In [33]:
get_e_k_hs_d_t(region=Region.REGION3, p=NPerson.FOUR).sum(), 28691.6793

(28691.679322596818, 28691.6793)

4地域

In [34]:
get_e_k_hs_d_t(region=Region.REGION4, p=NPerson.ONE).sum(), 205.3253 * 30 + 4415.4590

(10586.81678757223, 10575.218)

In [35]:
get_e_k_hs_d_t(region=Region.REGION4, p=NPerson.TWO).sum(), 272.2243 * 60 + 401.5186

(16734.97562574866, 16734.9766)

In [36]:
get_e_k_hs_d_t(region=Region.REGION4, p=NPerson.THREE).sum(), 97.7986 * 90 + 16099.8321

(24901.704140979084, 24901.7061)

In [37]:
get_e_k_hs_d_t(region=Region.REGION4, p=NPerson.FOUR).sum(), 27835.6615

(27835.66149708824, 27835.6615)

5地域

In [38]:
get_e_g_hs_d_t(region=Region.REGION5, p=NPerson.ONE).sum(), 200.6680 * 30 + 4420.0975

(10469.321613949627, 10440.1375)

In [39]:
get_e_g_hs_d_t(region=Region.REGION5, p=NPerson.TWO).sum(), 276.3005 * 60 - 117.8525

(16460.17887396076, 16460.177499999998)

In [40]:
get_e_g_hs_d_t(region=Region.REGION5, p=NPerson.THREE).sum(), 103.8316 * 90 + 15404.3532

(24749.19456524703, 24749.1972)

In [41]:
get_e_g_hs_d_t(region=Region.REGION5, p=NPerson.FOUR).sum()

27864.141671200367

6地域

In [42]:
get_e_g_hs_d_t(region=Region.REGION6, p=NPerson.ONE).sum(), 181.3259 * 30 + 3961.9807

(9425.830953756358, 9401.7577)

In [43]:
get_e_g_hs_d_t(region=Region.REGION6, p=NPerson.TWO).sum(), 249.7700 * 60 - 144.6684

(14841.533190864859, 14841.5316)

In [44]:
get_e_g_hs_d_t(region=Region.REGION6, p=NPerson.THREE).sum(), 93.9853 * 90 + 13875.9537

(22334.63400116704, 22334.6307)

In [45]:
get_e_g_hs_d_t(region=Region.REGION6, p=NPerson.FOUR).sum(), 25154.1941

(25154.194116361934, 25154.1941)

7地域

In [46]:
get_e_g_hs_d_t(region=Region.REGION7, p=NPerson.ONE).sum(), 165.2424 * 30 + 3542.3920

(8518.947208951698, 8499.664)

In [47]:
get_e_g_hs_d_t(region=Region.REGION7, p=NPerson.TWO).sum(), 227.0405 * 60 - 165.4938

(13456.938053881042, 13456.9362)

In [48]:
get_e_g_hs_d_t(region=Region.REGION7, p=NPerson.THREE).sum(), 85.4735 * 90 + 12575.5402

(20268.154003897933, 20268.1552)

In [49]:
get_e_g_hs_d_t(region=Region.REGION7, p=NPerson.FOUR).sum(), 22832.3586

(22832.35860391573, 22832.3586)

8地域

In [50]:
get_e_g_hs_d_t(region=Region.REGION8, p=NPerson.ONE).sum(), 130.0332 * 30 + 2771.4274

(6681.282680304047, 6672.4234)

In [51]:
get_e_g_hs_d_t(region=Region.REGION8, p=NPerson.TWO).sum(), 178.9496 * 60 - 163.5566

(10573.420626700175, 10573.4194)

In [52]:
get_e_g_hs_d_t(region=Region.REGION8, p=NPerson.THREE).sum(), 67.5958 * 90 + 9858.2861

(15941.909223571707, 15941.908099999999)

In [53]:
get_e_g_hs_d_t(region=Region.REGION8, p=NPerson.FOUR).sum(), 17969.7836

(17969.783584981727, 17969.7836)

月別の値の算出

In [54]:
def sum_monthly(x_d_t: np.ndarray):

    d_m = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    d_end = np.cumsum(d_m)
    d_str = np.insert(d_end[0: -1], 0, 0)

    x_d = np.sum(x_d_t, axis=1)
    
    return np.array([x_d[d_s: d_e].sum() for (d_s, d_e) in zip(d_str, d_end)])

In [55]:
result = np.array([
    [
        sum_monthly(get_e_k_hs_d_t(region=Region.REGION1, p=p)),
        sum_monthly(get_e_k_hs_d_t(region=Region.REGION2, p=p)),
        sum_monthly(get_e_k_hs_d_t(region=Region.REGION3, p=p)),
        sum_monthly(get_e_k_hs_d_t(region=Region.REGION4, p=p)),
        sum_monthly(get_e_g_hs_d_t(region=Region.REGION5, p=p)),
        sum_monthly(get_e_g_hs_d_t(region=Region.REGION6, p=p)),
        sum_monthly(get_e_g_hs_d_t(region=Region.REGION7, p=p)),
        sum_monthly(get_e_g_hs_d_t(region=Region.REGION8, p=p))
    ]
    for p
    in [NPerson.ONE, NPerson.TWO, NPerson.THREE, NPerson.FOUR]
]).round(1)

result.shape


(4, 8, 12)

In [56]:
with pd.ExcelWriter('dhw_table.xlsx') as writer:
    pd.DataFrame(result[0]).T.to_excel(writer, sheet_name='ONE')
    pd.DataFrame(result[1]).T.to_excel(writer, sheet_name='TWO')
    pd.DataFrame(result[2]).T.to_excel(writer, sheet_name='THREE')
    pd.DataFrame(result[3]).T.to_excel(writer, sheet_name='FOUR')
